***
# Setup
***

In [1]:
!pip install -r requirements.txt

import logging
#logger = logging.getLogger()
#logger.setLevel(logging.INFO)

import pandas as pd
import requests
import zipfile
import os
import shutil
from SPARQLWrapper import SPARQLWrapper, JSON, XML
import logging

from data_imports import *

***
# Import Data
***

In [2]:
df_api = import_kantonZH_api() 
df_wikidata = import_wikidata_kantonZH()
swisstopowikidata = import_swisstopowikidata_kantonZH()

In [3]:
print(df_api.shape)
df_api.head()

(10032, 4)


,BFS_NR,GEBIET_NAME,date,population
0,21,Adlikon,2018-12-31,707.0
1,131,Adliswil,2018-12-31,18681.0
2,241,Aesch ZH,2018-12-31,1555.0
3,1,Aeugst a.A.,2018-12-31,1979.0
4,2,Affoltern a.A.,2018-12-31,12201.0


In [4]:
print(df_wikidata.shape)
df_wikidata.head()

(815, 7)


,bfs_id,date,population,qualifier,refpublisher,refurl,wikidata_id
0,1,2017-01-01,1934,NormalRank,NA,https://statistik.zh.ch/internet/justiz_innere...,Q64945
1,1,2017-12-31,1941,NormalRank,http://www.wikidata.org/entity/Q285453,https://www.pxweb.bfs.admin.ch/pxweb/de/px-x-0...,Q64945
2,1,2018-12-31,1982,NormalRank,http://www.wikidata.org/entity/Q285453,https://www.bfs.admin.ch/bfs/de/home/statistik...,Q64945
3,2,2012-12-31,11276,NormalRank,NA,http://www.pxweb.bfs.admin.ch/Dialog/varval.as...,Q68290
4,2,2017-01-01,12080,PreferredRank,NA,https://statistik.zh.ch/internet/justiz_innere...,Q68290


In [5]:
print(swisstopowikidata.shape)
swisstopowikidata.head()

(162, 3)


,Name,bfs,wikidata_id
0,Stadel,100,Q67907
1,Obfelden,10,Q65942
2,Rüti (ZH),118,Q69752
3,Seegräben,119,Q67310
4,Ottenbach,11,Q67513


***

# Merge df_api with swisstopowikidata 

Aim: add wikidata qnumber to df_api 

Observation: Not all bfsnumbers can be linked to a qnumber. Reason: "Gemeindefusionen"
        
Decision: Only use bfsnumbers that are active today.           
***  

In [6]:
df_api_qnumber = pd.merge(df_api, swisstopowikidata, how='left', left_on=['BFS_NR'], right_on=['bfs'])
print(df_api_qnumber.shape)
df_api_qnumber.head()

(10032, 7)


,BFS_NR,GEBIET_NAME,date,population,Name,bfs,wikidata_id
0,21,Adlikon,2018-12-31,707.0,Adlikon,21.0,Q68356
1,131,Adliswil,2018-12-31,18681.0,Adliswil,131.0,Q68210
2,241,Aesch ZH,2018-12-31,1555.0,Aesch (ZH),241.0,Q68342
3,1,Aeugst a.A.,2018-12-31,1979.0,Aeugst am Albis,1.0,Q64945
4,2,Affoltern a.A.,2018-12-31,12201.0,Affoltern am Albis,2.0,Q68290


In [7]:
# view na in kantonZHapiANDswisstopowikidata
df_api_qnumber_na = df_api_qnumber[df_api_qnumber.isna().any(axis=1)].sort_values(by=['BFS_NR'])
print(df_api_qnumber_na.shape)
print(df_api_qnumber_na.BFS_NR.unique()) # !!! Warum sind für diese keine Q Wikinr. vorhanden?)
df_api_qnumber_na

(798, 7)
[ 36  42  44 132 133 134 140 142 171 174 175 179 217 222]


,BFS_NR,GEBIET_NAME,date,population,Name,bfs,wikidata_id
8024,36,Oberstammheim (bis 2018),1973-12-31,760.0,NaN,NaN,NaN
1160,36,Oberstammheim (bis 2018),2012-12-31,1125.0,NaN,NaN,NaN
3624,36,Oberstammheim (bis 2018),1998-12-31,1041.0,NaN,NaN,NaN
9256,36,Oberstammheim (bis 2018),1966-12-31,850.0,NaN,NaN,NaN
7496,36,Oberstammheim (bis 2018),1976-12-31,785.0,NaN,NaN,NaN
3800,36,Oberstammheim (bis 2018),1997-12-31,1042.0,NaN,NaN,NaN
984,36,Oberstammheim (bis 2018),2013-12-31,1142.0,NaN,NaN,NaN
7320,36,Oberstammheim (bis 2018),1977-12-31,788.0,NaN,NaN,NaN
4152,36,Oberstammheim (bis 2018),1995-12-31,1069.0,NaN,NaN,NaN
9432,36,Oberstammheim (bis 2018),1965-12-31,831.0,NaN,NaN,NaN


In [8]:
df_api_qnumber[df_api_qnumber['BFS_NR']==297]

,BFS_NR,GEBIET_NAME,date,population,Name,bfs,wikidata_id
11,297,Bauma,2018-12-31,4961.0,Bauma,297.0,Q67145
187,297,Bauma,2017-12-31,4925.0,Bauma,297.0,Q67145
363,297,Bauma,2016-12-31,4896.0,Bauma,297.0,Q67145
539,297,Bauma,2015-12-31,4837.0,Bauma,297.0,Q67145
715,297,Bauma,2014-12-31,4718.0,Bauma,297.0,Q67145
891,297,Bauma,2013-12-31,4650.0,Bauma,297.0,Q67145
1067,297,Bauma,2012-12-31,4569.0,Bauma,297.0,Q67145
1243,297,Bauma,2011-12-31,4550.0,Bauma,297.0,Q67145
1419,297,Bauma,2010-12-31,4522.0,Bauma,297.0,Q67145
1595,297,Bauma,2009-12-31,4516.0,Bauma,297.0,Q67145


In [9]:
df_api_qnumber[df_api_qnumber['BFS_NR']==171]

,BFS_NR,GEBIET_NAME,date,population,Name,bfs,wikidata_id
12,171,Bauma (bis 2014),2018-12-31,NaN,NaN,NaN,NaN
188,171,Bauma (bis 2014),2017-12-31,NaN,NaN,NaN,NaN
364,171,Bauma (bis 2014),2016-12-31,NaN,NaN,NaN,NaN
540,171,Bauma (bis 2014),2015-12-31,NaN,NaN,NaN,NaN
716,171,Bauma (bis 2014),2014-12-31,4367.0,NaN,NaN,NaN
892,171,Bauma (bis 2014),2013-12-31,4297.0,NaN,NaN,NaN
1068,171,Bauma (bis 2014),2012-12-31,4218.0,NaN,NaN,NaN
1244,171,Bauma (bis 2014),2011-12-31,4199.0,NaN,NaN,NaN
1420,171,Bauma (bis 2014),2010-12-31,4160.0,NaN,NaN,NaN
1596,171,Bauma (bis 2014),2009-12-31,4150.0,NaN,NaN,NaN


In [10]:
df_api_qnumber[df_api_qnumber['BFS_NR']==179]

,BFS_NR,GEBIET_NAME,date,population,Name,bfs,wikidata_id
140,179,Sternenberg (bis 2014),2018-12-31,NaN,NaN,NaN,NaN
316,179,Sternenberg (bis 2014),2017-12-31,NaN,NaN,NaN,NaN
492,179,Sternenberg (bis 2014),2016-12-31,NaN,NaN,NaN,NaN
668,179,Sternenberg (bis 2014),2015-12-31,NaN,NaN,NaN,NaN
844,179,Sternenberg (bis 2014),2014-12-31,351.0,NaN,NaN,NaN
1020,179,Sternenberg (bis 2014),2013-12-31,353.0,NaN,NaN,NaN
1196,179,Sternenberg (bis 2014),2012-12-31,351.0,NaN,NaN,NaN
1372,179,Sternenberg (bis 2014),2011-12-31,351.0,NaN,NaN,NaN
1548,179,Sternenberg (bis 2014),2010-12-31,362.0,NaN,NaN,NaN
1724,179,Sternenberg (bis 2014),2009-12-31,366.0,NaN,NaN,NaN


In [11]:
df_api_qnumber_dropna = pd.merge(df_api, swisstopowikidata, how='left', left_on=['BFS_NR'], right_on=['bfs']).dropna()
print(df_api_qnumber_dropna.shape)
df_api_qnumber_dropna.head()

(9234, 7)


,BFS_NR,GEBIET_NAME,date,population,Name,bfs,wikidata_id
0,21,Adlikon,2018-12-31,707.0,Adlikon,21.0,Q68356
1,131,Adliswil,2018-12-31,18681.0,Adliswil,131.0,Q68210
2,241,Aesch ZH,2018-12-31,1555.0,Aesch (ZH),241.0,Q68342
3,1,Aeugst a.A.,2018-12-31,1979.0,Aeugst am Albis,1.0,Q64945
4,2,Affoltern a.A.,2018-12-31,12201.0,Affoltern am Albis,2.0,Q68290


In [12]:
# check
if df_api_qnumber_na.shape[0] + df_api_qnumber_dropna.shape[0] == df_api_qnumber.shape[0]:
    print("check ok")

check ok


***
# Merge df_api with wikidata TODO !!!! MESSY !!!


Aim: Check population and ... ???

***

In [13]:
df_api_qnumber_dropna['check'] = df_api_qnumber_dropna['date'].astype(str) + '---' + df_api['BFS_NR'].astype(str)
df_wikidata['check'] = df_wikidata['date'].astype(str) + '---' + df_wikidata['bfs_id'].astype(str)

In [14]:
df_api_sel_f = df_api_qnumber_dropna[(df_api_qnumber_dropna['check'].isin(df_wikidata['check']) == False)]
df_api_sel_f.to_csv("test.tsv", header = True, sep='\t')
print(df_api_sel_f.columns)
print(df_api_sel_f.shape)
df_api_sel_f.head()

Index(['BFS_NR', 'GEBIET_NAME', 'date', 'population', 'Name', 'bfs',
       'wikidata_id', 'check'],
      dtype='object')
(8871, 8)


,BFS_NR,GEBIET_NAME,date,population,Name,bfs,wikidata_id,check
138,292,Stammheim,2018-12-31,2740.0,Stammheim,292.0,Q60322693,2018-12-31---292
213,294,Elgg,2017-12-31,4903.0,Elgg,294.0,Q67137,2017-12-31---294
314,292,Stammheim,2017-12-31,2741.0,Stammheim,292.0,Q60322693,2017-12-31---292
352,21,Adlikon,2016-12-31,662.0,Adlikon,21.0,Q68356,2016-12-31---21
353,131,Adliswil,2016-12-31,18651.0,Adliswil,131.0,Q68210,2016-12-31---131


In [15]:
df_api_sel_t = df_api_qnumber_dropna[(df_api_qnumber_dropna['check'].isin(df_wikidata['check']) == True)]
print(df_api_sel_t.columns)
print(df_api_sel_t.shape)
df_api_sel_t.head()

Index(['BFS_NR', 'GEBIET_NAME', 'date', 'population', 'Name', 'bfs',
       'wikidata_id', 'check'],
      dtype='object')
(363, 8)


,BFS_NR,GEBIET_NAME,date,population,Name,bfs,wikidata_id,check
0,21,Adlikon,2018-12-31,707.0,Adlikon,21.0,Q68356,2018-12-31---21
1,131,Adliswil,2018-12-31,18681.0,Adliswil,131.0,Q68210,2018-12-31---131
2,241,Aesch ZH,2018-12-31,1555.0,Aesch (ZH),241.0,Q68342,2018-12-31---241
3,1,Aeugst a.A.,2018-12-31,1979.0,Aeugst am Albis,1.0,Q64945,2018-12-31---1
4,2,Affoltern a.A.,2018-12-31,12201.0,Affoltern am Albis,2.0,Q68290,2018-12-31---2


In [28]:
df_api_wikidata = pd.merge(df_api_sel_t, df_wikidata, how='left', left_on=['check'], right_on=['check'])
# df_api instead of df_api_sel_t ???
print(df_api_wikidata.columns)
print(df_api_wikidata.shape)
print(df_api_wikidata.shape)
df_api_wikidata[["Name", "BFS_NR", "wikidata_id_y", "date_x", "date_y", "population_x", "population_y", "refurl"]].sort_values(by=['BFS_NR'])

KeyError: 'check'

In [18]:
# Find rows where check is duplicated
duplicate = df_api_wikidata[df_api_wikidata.duplicated(['check'])]
print(duplicate.shape)
duplicate.sort_values(by=['check'])


(23, 15)


,BFS_NR,GEBIET_NAME,date_x,population_x,Name,bfs,wikidata_id_x,check,bfs_id,date_y,population_y,qualifier,refpublisher,refurl,wikidata_id_y
326,102,Weiach,2017-12-31,1750.0,Weiach,102.0,Q70088,2017-12-31---102,102,2017-12-31,1750,PreferredRank,NA,https://www.weiach.ch/page/13,Q70088
325,102,Weiach,2017-12-31,1750.0,Weiach,102.0,Q70088,2017-12-31---102,102,2017-12-31,1756,NormalRank,http://www.wikidata.org/entity/Q285453,https://www.pxweb.bfs.admin.ch/pxweb/de/px-x-0...,Q70088
290,178,Russikon,2017-12-31,4355.0,Russikon,178.0,Q67957,2017-12-31---178,178,2017-12-31,4367,NormalRank,http://www.wikidata.org/entity/Q285453,https://www.pxweb.bfs.admin.ch/pxweb/de/px-x-0...,Q67957
336,181,Wila,2017-12-31,1975.0,Wila,181.0,Q66752,2017-12-31---181,181,2017-12-31,1975,NormalRank,http://www.wikidata.org/entity/Q285453,https://www.pxweb.bfs.admin.ch/pxweb/de/px-x-0...,Q66752
311,248,Uitikon,2017-12-31,4239.0,Uitikon,248.0,Q66678,2017-12-31---248,248,2017-12-31,4245,NormalRank,http://www.wikidata.org/entity/Q285453,https://www.pxweb.bfs.admin.ch/pxweb/de/px-x-0...,Q66678
233,295,Horgen,2017-12-31,22411.0,Horgen,295.0,Q68286,2017-12-31---295,295,2017-12-31,22476,NormalRank,NA,NA,Q68286
236,32,Humlikon,2017-12-31,486.0,Humlikon,32.0,Q68447,2017-12-31---32,32,2017-12-31,486,NormalRank,NA,https://statistik.zh.ch/internet/justiz_innere...,Q68447
237,32,Humlikon,2017-12-31,486.0,Humlikon,32.0,Q68447,2017-12-31---32,32,2017-12-31,486,NormalRank,NA,https://statistik.zh.ch/internet/justiz_innere...,Q68447
322,70,Wasterkingen,2017-12-31,574.0,Wasterkingen,70.0,Q67882,2017-12-31---70,70,2017-12-31,574,NormalRank,http://www.wikidata.org/entity/Q285453,https://www.pxweb.bfs.admin.ch/pxweb/de/px-x-0...,Q67882
323,70,Wasterkingen,2017-12-31,574.0,Wasterkingen,70.0,Q67882,2017-12-31---70,70,2017-12-31,574,NormalRank,NA,https://statistik.zh.ch/internet/justiz_innere...,Q67882


In [19]:
# check
if duplicate.shape[0] + df_api_sel_t.shape[0] == df_api_wikidata.shape[0]:
    print("check ok")

check ok


In [26]:
# should be 162
print(len(df_api_wikidata.BFS_NR.unique())) # one is missing

print(len(df_api.BFS_NR.unique())) #too many because of "Gemeindefusion"

print(len(df_api_qnumber_dropna.BFS_NR.unique())) #ok

print(len(df_api_sel_t.BFS_NR.unique())) # one is missing


161
176
162
161
